# Welcome to DFL-Colab!

This is an adapted version of the DFL for Google Colab.


# Overview
*   Extractor works in full functionality.
*   Training can work without preview.
*   Merger works in full functionality.
*   You can import/export workspace with your Google Drive.
*   Import/export and another manipulations with workspace you can do in "Manage workspace" block
*   Google Colab machine active for 12 hours. DFL-Colab makes a backup of your workspace in training mode.
*   Google does not like long-term heavy calculations. Therefore, for training more than two sessions in a row, use two Google accounts. It is recommended to split your training over 2 accounts, but you can use one Google Drive account to store your workspace.



## Prevent random disconnects

This cell runs JS code to automatic reconnect to runtime.

In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

## Check GPU

*   Google Colab can provide you with one of Tesla graphics cards: K80, T4, P4 or P100
*   Here you can check the model of GPU before using DeepFaceLab



In [ ]:
!nvidia-smi

## Install or update DeepFaceLab

* Install or update DeepFAceLab directly from Github
* Requirements install is automatically
* Automatically sets timer to prevent random disconnects
* "Download FFHQ" option means to download high quality FFHQ dataset instead of CelebA. FFHQ takes up more memory, so it will take longer to download than CelebA. It is recommended to enable this option if you are doing pretrain.

In [ ]:
#@title Install or update DeepFaceLab from Github

Mode = "install" #@param ["install", "update", "remove"]
Repository = "Iperov" #@param ["Iperov", "faceshiftlabs", "Cioscos"]
Branch_name = "" #@param {type:"string"}
Download_pretrain_datasets = False #@param {type: "boolean"}
Download_FFHQ = False #@param {type:"boolean"}


pretrain_link = "https://github.com/chervonij/DFL-Colab/releases/download/"
pretrain_link = pretrain_link+"pretrain_GenericFFHQ/pretrain_FFHQ.zip" if Download_FFHQ else pretrain_link+"pretrain-CelebA/pretrain_CelebA.zip"

from pathlib import Path
if Mode == 'install':
  git_cmd = ''
  if Branch_name: git_cmd += f"-b {Branch_name} "
  if Repository == 'Cioscos':
    answer = input('This is an experimental Repository with different SAEHD file.\nDo yuo want to continue? (y/n)').lower()
    while answer not in ["y", "n"]:
      answer = input('Invalid input\nThis is an experimental Repository with different SAEHD file.\nDo yuo want to continue?').lower()
  
  if answer == 'n':
    exit(1)

  git_cmd += f"https://github.com/{Repository.lower()}/DeepFaceLab.git"
  !git clone $git_cmd
  if Repository != 'Iperov':
    !git clone https://github.com/aidenlt/temporary-colab-requirements-fix.git

  # fix linux warning
  # /usr/lib/python3.6/multiprocessing/semaphore_tracker.py:143: UserWarning: semaphore_tracker: There appear to be 1 leaked semaphores to clean up at shutdown
  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "rt")
  data = fin.read()
  data = data.replace('if cache:', 'if False:')
  fin.close()

  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "wt")
  fin.write(data)
  fin.close()
elif Mode == 'update':
  %cd /content/DeepFaceLab
  !git pull
else:
  answer = input("Are you sure you want to delete DFL folder? (y/n)").lower()
  while answer not in ["y", "n"]:
    print("Invalid input")
    answer = input("Are you sure you want to delete DFL folder? (y/n)").lower()
  if answer == "y":
    %cd /content
    %rm -r DeepFaceLab/
    print("Done!")
  else:
    print("Cancelled...")

if Mode != 'remove':
  !pip uninstall -y tensorflow
  if Repository != 'Iperov':
    !pip install -r /content/temporary-colab-requirements-fix/requirements-colab.txt
  else:
    !pip install -r /content/DeepFaceLab/requirements-colab.txt
  !pip install --upgrade scikit-image
  !apt-get install cuda-10-0

  if Download_pretrain_datasets:

    if not Path("/content/pretrain").exists():
      print("Downloading Pretrain faceset ... ")
      !wget -q --no-check-certificate -r $pretrain_link -O /content/pretrain_faceset.zip
      !mkdir /content/pretrain
      !unzip -q /content/pretrain_faceset.zip -d /content/pretrain/
      !rm /content/pretrain_faceset.zip

    if not Path("/content/pretrain_Q96").exists():
      print("Downloading Q96 pretrained model ...")
      !wget -q --no-check-certificate -r 'https://github.com/chervonij/DFL-Colab/releases/download/Q96_model_pretrained/Q96_model_pretrained.zip' -O /content/pretrain_Q96.zip
      !mkdir /content/pretrain_Q96
      !unzip -q /content/pretrain_Q96.zip -d /content/pretrain_Q96/
      !rm /content/pretrain_Q96.zip

  if not Path("/content/workspace").exists():
    !mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model

  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
   function ClickConnect(){
     btn = document.querySelector("colab-connect-button")
     if (btn != null){
       console.log("Click colab-connect-button");
       btn.click()
     }

     btn = document.getElementById('ok')
     if (btn != null){
       console.log("Click reconnect");
       btn.click()
     }
   }

  setInterval(ClickConnect,60000)
'''))

print("\nDone!")

## Manage workspace



*   You can import/export workspace or individual data, like model files with Google Drive
*   Also, you can use HFS (HTTP Fileserver) for directly import/export you workspace from your computer
*   You can clear all workspace or delete part of it



In [ ]:
#@title Import from Drive

Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "models"]
Archive_name = "workspace.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  print("Copying...")
  !cp $copy_cmd
  print("Unzipping...")
  !unzip $unzip_cmd    
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")
  
print("Done!")



In [ ]:
#@title Export to Drive { form-width: "30%" }
Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "merged", "merged_mask", "models", "result video", "result_mask video"]
Archive_name = "workspace.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  zip_cmd="-r -q "+Archive_name+" "
  
  %cd $path
  zip_cmd+=mode
  print("Zipping...")
  !zip $zip_cmd
  copy_cmd = " "+Archive_name+"  /content/drive/My\ Drive/"
  print("Copying...")
  !cp $copy_cmd
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "merged":
  zip_and_copy("/content/workspace/data_dst", "merged")
elif Mode == "merged_mask":
  zip_and_copy("/content/workspace/data_dst", "merged_mask")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")
elif Mode == "result video":
  !cp /content/workspace/result.mp4 /content/drive/My\ Drive/
elif Mode == "result_mask video":
  !cp /content/workspace/result_mask.mp4 /content/drive/My\ Drive/
  
print("Done!")


In [ ]:
#@title Import from URL{ form-width: "30%", display-mode: "form" }
URL = "http://" #@param {type:"string"}
Mode = "unzip to content" #@param ["unzip to content", "unzip to content/workspace", "unzip to content/workspace/data_src", "unzip to content/workspace/data_src/aligned", "unzip to content/workspace/data_dst", "unzip to content/workspace/data_dst/aligned", "unzip to content/workspace/model", "download to content/workspace"]

import urllib.request
from pathlib import Path

def unzip(zip_path, dest_path):
  unzip_cmd = " unzip -q " + zip_path + " -d "+dest_path
  print('Unzipping...')
  !$unzip_cmd  
  rm_cmd = "rm "+dest_path + url_path.name
  !$rm_cmd
  

if Mode == "unzip to content":
  dest_path = "/content/"
elif Mode == "unzip to content/workspace":
  dest_path = "/content/workspace/"
elif Mode == "unzip to content/workspace/data_src":
  dest_path = "/content/workspace/data_src/"
elif Mode == "unzip to content/workspace/data_src/aligned":
  dest_path = "/content/workspace/data_src/aligned/"
elif Mode == "unzip to content/workspace/data_dst":
  dest_path = "/content/workspace/data_dst/"
elif Mode == "unzip to content/workspace/data_dst/aligned":
  dest_path = "/content/workspace/data_dst/aligned/"
elif Mode == "unzip to content/workspace/model":
  dest_path = "/content/workspace/model/"
elif Mode == "download to content/workspace":
  dest_path = "/content/workspace/"

if not Path("/content/workspace").exists():
  cmd = "mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model"
  !$cmd

url_path = Path(URL)
print('Downloading...')
urllib.request.urlretrieve ( URL, dest_path + url_path.name )

if (url_path.suffix == ".zip") and (Mode!="download to content/workspace"):
  unzip(dest_path + url_path.name, dest_path)

print("Done!")

In [ ]:
#@title Export to URL
URL = "http://" #@param {type:"string"}
Mode = "upload workspace" #@param ["upload workspace", "upload data_src", "upload data_dst", "upload data_src aligned", "upload data_dst aligned", "upload merged", "upload model", "upload result video"]

cmd_zip = "zip -r -q "

def run_cmd(zip_path, curl_url):
  cmd_zip = "zip -r -q "+zip_path
  cmd_curl = "curl --silent -F "+curl_url+" -D out.txt > /dev/null"
  print('Zipping...')
  !$cmd_zip
  print('Copying...')
  !$cmd_curl


if Mode == "upload workspace":
  %cd "/content"
  run_cmd("workspace.zip workspace/","'data=@/content/workspace.zip' "+URL)
elif Mode == "upload data_src":
  %cd "/content/workspace"
  run_cmd("data_src.zip data_src/", "'data=@/content/workspace/data_src.zip' "+URL)
elif Mode == "upload data_dst":
  %cd "/content/workspace"
  run_cmd("data_dst.zip data_dst/", "'data=@/content/workspace/data_dst.zip' "+URL)
elif Mode == "upload data_src aligned":
  %cd "/content/workspace"
  run_cmd("data_src_aligned.zip data_src/aligned", "'data=@/content/workspace/data_src_aligned.zip' "+URL )
elif Mode == "upload data_dst aligned":
  %cd "/content/workspace"
  run_cmd("data_dst_aligned.zip data_dst/aligned/", "'data=@/content/workspace/data_dst_aligned.zip' "+URL)
elif Mode == "upload merged":
  %cd "/content/workspace/data_dst"
  run_cmd("merged.zip merged/","'data=@/content/workspace/data_dst/merged.zip' "+URL )
elif Mode == "upload model":
  %cd "/content/workspace"
  run_cmd("model.zip model/", "'data=@/content/workspace/model.zip' "+URL)
elif Mode == "upload result video":
  %cd "/content/workspace"
  run_cmd("result.zip result.mp4", "'data=@/content/workspace/result.zip' "+URL)
  
  
!rm *.zip

%cd "/content"
print("Done!")

In [ ]:
#@title Delete and recreate
Mode = "Delete and recreate workspace" #@param ["Delete and recreate workspace", "Delete models", "Delete data_src", "Delete data_src aligned", "Delete data_src video", "Delete data_dst", "Delete data_dst aligned", "Delete merged frames"]

%cd "/content" 

if Mode == "Delete and recreate workspace":
  cmd = "rm -r /content/workspace ; mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model"  
elif Mode == "Delete models":
  cmd = "rm -r /content/workspace/model/*"
elif Mode == "Delete data_src":
  cmd = "rm /content/workspace/data_src/*.png || rm -r /content/workspace/data_src/*.jpg"
elif Mode == "Delete data_src aligned":
  cmd = "rm -r /content/workspace/data_src/aligned/*"
elif Mode == "Delete data_src video":
  cmd = "rm -r /content/workspace/data_src.*"
elif Mode == "Delete data_dst":
  cmd = "rm /content/workspace/data_dst/*.png || rm /content/workspace/data_dst/*.jpg"
elif Mode == "Delete data_dst aligned":
  cmd = "rm -r /content/workspace/data_dst/aligned/*"
elif Mode == "Delete merged frames":
  cmd = "rm -r /content/workspace/data_dst/merged; rm -r /content/workspace/data_dst/merged_mask"
  
answer = input("Are you sure you want to " + Mode + "? (y/n)")
while answer not in ["y", "n"]:
  print("Invalid input")
  answer = input("Are you sure you want to " + Mode + "? (y/n)")

if answer == "y":
  !$cmd
  print("Done!")
else:
  print("Cancelled...")

## Extract, sorting and faceset tools
* Extract frames for SRC or DST video.
* Denoise SRC or DST video. "Factor" param set intesity of denoising
* Detect and align faces. If you need, you can get frames with debug landmarks.
* Export workspace to Google Drive after extract and sort it manually (In "Manage Workspace" block)
* You can enhance your facesets with DFL FacesetEnhancer.
* Resize faceset to your model resolution. Since Colab doesn't have a powerful CPU, resizing samples during training increases iteration time. Faceset resize reduces iteration time by about 2x times. Don't forget to keep save original faceset on your PC.
* Pack or unpack facesets with DFL packing tool.
* Apply or remove trained XSeg model to the extracted faces.
* Recommended for use, Generic XSeg model for auto segmentation.

In [ ]:
#@title Extract frames
Video = "data_src" #@param ["data_src", "data_dst"]

%cd "/content"

cmd = "DeepFaceLab/main.py videoed extract-video"

if Video == "data_dst":
  cmd+= " --input-file workspace/data_dst.* --output-dir workspace/data_dst/"
else:
  cmd+= " --input-file workspace/data_src.* --output-dir workspace/data_src/"
  
!python $cmd

In [ ]:
#@title Denoise frames
Data = "data_src" #@param ["data_src", "data_dst"]
Factor = 1 #@param {type:"slider", min:1, max:20, step:1}

cmd = "DeepFaceLab/main.py videoed denoise-image-sequence --input-dir workspace/"+Data+" --factor "+str(Factor)

%cd "/content"
!python $cmd

In [ ]:
#@title Detect faces
Data = "data_src" #@param ["data_src", "data_dst"]
Detector = "S3FD" #@param ["S3FD", "S3FD (whole face)"]
Debug = False #@param {type:"boolean"}

detect_type = "s3fd"
dbg = " --output-debug" if Debug else " --no-output-debug"

folder = "workspace/"+Data
folder_aligned = folder+"/aligned"

cmd = "DeepFaceLab/main.py extract --input-dir "+folder+" --output-dir "+folder_aligned
cmd+=" --detector "+detect_type+" --force-gpu-idxs 0"+dbg

if "whole face" in Detector:
  cmd+=" --face-type whole_face" 
%cd "/content"
!python $cmd

In [ ]:
#@title Sort aligned
Data = "data_src" #@param ["data_src", "data_dst"]
sort_type = "hist" #@param ["blur", "motion-blur", "face-yaw", "face-pitch", "face-source-rect-size", "hist", "hist-dissim", "brightness", "hue", "black", "origname", "oneface", "final-by-blur", "final-by-size", "absdiff"]

cmd = "DeepFaceLab/main.py sort --input-dir workspace/"+Data+"/aligned --by "+sort_type

%cd "/content"
!python $cmd

In [ ]:
#@title Faceset Enhancer
Data = "data_src" #@param ["data_src", "data_dst"]

data_path = "/content/workspace/"+Data+"/aligned"
cmd = "/content/DeepFaceLab/main.py facesettool enhance --input-dir "+data_path
!python $cmd

In [ ]:
#@title Resize faceset
Data = "data_src" #@param ["data_src", "data_dst"]

cmd = "/content/DeepFaceLab/main.py facesettool resize --input-dir /content/workspace/" + \
      f"{Data}/aligned"

!python $cmd

In [ ]:
#@title Pack/Unpack aligned faceset

Folder = "data_src" #@param ["data_src", "data_dst"]
Mode = "unpack" #@param ["pack", "unpack"]

cmd = "/content/DeepFaceLab/main.py util --input-dir /content/workspace/" + \
      f"{Folder}/aligned --{Mode}-faceset"

!python $cmd

In [ ]:
#@title Apply or remove XSeg mask to the faces
Mode = "Apply mask" #@param ["Apply mask", "Remove mask"]
Data = "data_src" #@param ["data_src", "data_dst"]
GenericXSeg = True #@param {type:"boolean"}

from pathlib import Path
mode_arg = 'apply' if Mode == "Apply mask" else 'remove'

if GenericXSeg and not Path('/content/GenericXSeg').exists():
  print('Downloading Generic XSeg model ... ')
  xseg_link = 'https://github.com/chervonij/DFL-Colab/releases/download/GenericXSeg/GenericXSeg.zip'
  !mkdir /content/GenericXSeg
  !wget -q --no-check-certificate -r $xseg_link -O /content/GenericXSeg.zip
  !unzip -q /content/GenericXSeg.zip -d /content/GenericXSeg/
  !rm /content/GenericXSeg.zip

main_path = '/content/DeepFaceLab/main.py'
data_path = f'/content/workspace/{Data}/aligned'
model_path = '/content/workspace/model' if not GenericXSeg else '/content/GenericXSeg'

cmd = f'{main_path} xseg {mode_arg} --input-dir {data_path} '
cmd += f'--model-dir {model_path}' if mode_arg == 'apply' else ''

!python $cmd

## Train model

* Choose your model type, but SAEHD is recommend for everyone
* Set model options on output field
* You can see preview manually, if go to model folder in filemanager and double click on preview.jpg file
* Your workspace will be archived and upload to mounted Drive after 11 hours from start session
* If you select "Backup_every_hour" option, your workspace will be backed up every hour.
* Also, you can export your workspace manually in "Manage workspace" block
* "Silent_Start" option provides to automatically start with best GPU and last used model. 

In [ ]:
#@title Uptime
import psutil, os, time

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()
print(f'Session in running from {int(uptime / 3600)} hours')

In [ ]:
#@title Training
Model = "SAEHD" #@param ["SAEHD", "AMP", "Quick96", "XSeg"]
Train_only_src = False #@param {type:"boolean"}
Archive_name_backup = "workspace.zip" #@param {type:"string"}
Export_model_only = False #@param {type:"boolean"}
Backup_every_specified_time = True #@param {type:"boolean"}
backup_time =   1#@param {type:"number"}
Silent_Start = True #@param {type:"boolean"}

%cd "/content"

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive')

import psutil, os, time

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()

if Export_model_only:
  if os.path.exists(Archive_name_backup) and not os.path.exists(Archive_name_backup.replace('.zip', '_model.zip')):
    os.system(f"rm {Archive_name_backup}")
    print('Full workspace archive deleted\nCreating model archive...')
    os.system('zip -r -q '+Archive_name_backup.replace('.zip', '_model.zip')+' workspace/model')
    print("Archive created!")
  elif not os.path.exists(Archive_name_backup) and not os.path.exists(Archive_name_backup.replace('.zip', '_model.zip')):
    print('Creating model archive...')
    os.system('zip -r -q '+Archive_name_backup.replace('.zip', '_model.zip')+' workspace/model')
  else:
    print('Archive exists')


if Backup_every_specified_time and not Export_model_only:
  if not os.path.exists(Archive_name_backup):
    if os.path.exists(Archive_name_backup.replace('.zip', '_model.zip')):
      print('Removing model archive...')
      os.system(f"rm {Archive_name_backup.replace('.zip', '_model.zip')}")
    print("Creating workspace archive ...")
    os.system('zip -r -q '+Archive_name_backup+' workspace')
    print("Archive created!")
  else:
    print("Archive exists!")

if Backup_every_specified_time:
  print("Time to end session: "+str(round((43200-uptime)/3600))+" hours")
  backup_time = str(backup_time * 3600)
  if not Export_model_only:
    backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -r -q "+Archive_name_backup+" workspace/model'); os.system('cp /content/"+Archive_name_backup+" /content/drive/My\ Drive/'); print('Backed up!') \""
  else:
    backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -r -q "+Archive_name_backup.replace('.zip', '_model.zip')+" workspace/model'); os.system('cp /content/"+Archive_name_backup.replace('.zip', '_model.zip') +" /content/drive/My\ Drive/'); print('Backed up!') \""
elif (round(39600-uptime) > 0):
  print("Time to backup: "+str(round((39600-uptime)/3600))+" hours")
  backup_time = str(round(39600-uptime))
  if not Export_model_only:
    backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -r -q "+Archive_name_backup+" workspace'); os.system('cp /content/"+Archive_name_backup+" /content/drive/My\ Drive/'); print('Backed up!') \""
  else:
    backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -r -q "+Archive_name_backup.replace('.zip', '_model.zip')+" workspace/model'); os.system('cp /content/"+Archive_name_backup.replace('.zip', '_model.zip') +" /content/drive/My\ Drive/'); print('Backed up!') \""
else:
  print("Session expires in less than an hour.")
  backup_cmd = ""

if Train_only_src:
  if Model != 'AMP':
    answer = ''
    print('The src-src training is only for AMP model. Do you want to continue anyway? (y/n)')
    while True:
      answer = input()
      if answer == 'y':
        cmd = "DeepFaceLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_src/aligned --pretraining-data-dir pretrain --model-dir workspace/model --model "+Model
      elif answer == 'n':
        cmd = "DeepFaceLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain --model-dir workspace/model --model "+Model
      else:
        print('Wrong value! Insert y or n:')
      if answer in ['y', 'n']:
        break
  else:
    cmd = "DeepFaceLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_src/aligned --pretraining-data-dir pretrain --model-dir workspace/model --model "+Model
else:
  cmd = "DeepFaceLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain --model-dir workspace/model --model "+Model

if Model == "Quick96":
  cmd+= " --pretrained-model-dir pretrain_Q96"

if Silent_Start:
  cmd+= " --silent-start"

if (backup_cmd != ""):
  train_cmd = (cmd+backup_cmd)
else:
  train_cmd = (cmd)

%env TCMALLOC_LARGE_ALLOC_REPORT_THRESHOLD = 10737418240 
!python $train_cmd

## Merge frames

In [ ]:
#@title Merge
Model = "SAEHD" #@param ["SAEHD", "AMP", "Quick96" ]

cmd = "DeepFaceLab/main.py merge --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --output-mask-dir workspace/data_dst/merged_mask --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

%cd "/content"
!python $cmd

In [ ]:
#@title Get result video 
Mode = "result video" #@param ["result video", "result_mask video"]
Bitrate = 25 #@param {type:"integer"}
Copy_to_Drive = True #@param {type:"boolean"}

cmd = "DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4 --include-audio --bitrate " + str(Bitrate)
cmd_mask = "DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged_mask --output-file workspace/result_mask.mp4 --reference-file workspace/data_dst.mp4 --bitrate " +str(Bitrate)

if Mode == "result video":
  !python $cmd
  if Copy_to_Drive:
    !cp /content/workspace/result.mp4 /content/drive/My\ Drive/
elif Mode == "result_mask video":
  !python $cmd_mask
  if Copy_to_Drive:
    !cp /content/workspace/result_mask.mp4 /content/drive/My\ Drive/
